# Selenium ile kariyer.net verilerini çekme

Selenium web sayfaları üzerinden otomasyon, test senaryoları yazmayı sağlayan bir kütüphanedir. Python dışında pek çok programlama dili ile kullanılabilir. Burada Selenium kullanma sebebim kariyer.net web sayfasının dinamik bir arayüzü olması sebebiyle sayfa açıldıktan sonra ekrana yansayacak verileri alabilmektir. Bu sayede daha çok veriyi eksiksiz ve doğru bir şekilde erişebilmiş olacağım.

In [3]:
#Gerekli fonksiyonları import edelim
from dfply import * #dplyr python portu
import time
import random
import pandas as pd
from bs4 import BeautifulSoup # daha ilkel bir html parse kutuphanesi
from selenium import webdriver

driver_path = "C:/webdriver/chromedriver.exe"#webdriver.exe konumunu göstermeliyiz. 
driver = webdriver.Chrome(driver_path) 
#
#
from selenium.webdriver.common.action_chains import ActionChains
def nextpage():
    ActionChains(driver).move_to_element(driver.find_element_by_id('lnkNextPage')).perform()
    driver.find_element_by_id('lnkNextPage').click()
    

In [4]:
urls=['https://www.kariyer.net/is-ilani/cemobsan-metal-ve-plastik-sanayi-ticaret-limited-s-ihracat-uzmani-2476220',
'https://www.kariyer.net/is-ilani/dogan-metalurji-makina-ve-ins-taahut-sanve-ticaret-dijital-iletisim-uzmani-2476214',
'https://www.kariyer.net/is-ilani/mitsuba-teklas-turkey-otomotiv-a-s-bakim-sorumlusu-2476147']
pozisyon=[]
city=[]
firma=[]
metin=[]
b_sayisi=[]
g_sayisi=[]
sure=[]
link=[]
KriterDF=[]
DT=None

In [5]:
for url in urls:
    driver.get(url)
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,700)")
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,1000)")
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0,1300)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,1600)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,1800)")
    time.sleep(0.2)
    driver.execute_script("window.scrollTo(0,2300)")
    time.sleep(4)
    driver.execute_script("window.scrollTo(0,1000)")  
    kriterler = {
        "bir": driver.find_elements_by_class_name("col-4"),
        "iki": driver.find_elements_by_class_name("col-12")
    }
    try: 
        pozisyon = pozisyon + [driver.find_element_by_id("jobTitle").text]
    except:
        pozisyon.append(None)
    try:
        city  = city  + [driver.find_element_by_id("jobCity").text]
    except:
        city.append(None)
    try:
        firma = firma + [driver.find_element_by_id("jobCompany").text]
    except:
        firma.append(None)
    try:
        metin = metin + [driver.find_element_by_id("divIlanSablon").text]
    except:
        metin.append(None)
    try:
        b_sayisi=b_sayisi+[driver.find_element_by_xpath("//*[contains(@class, 'basvuru-sayisi')]").find_element_by_css_selector('strong').text]
    except:
        b_sayisi.append(None)
    try:    
        g_sayisi=g_sayisi+[driver.find_element_by_xpath("//*[contains(@class, 'goruntulenme-sayisi')]").find_element_by_css_selector('strong').text]
    except:
        g_sayisi.append(None)
    try:
        sure=sure+[driver.find_element_by_xpath("//*[contains(@class, 'sure')]").find_element_by_css_selector('strong').text]
    except: 
        sure.append(None)
    
    # veriye ait tablo bilgilerini çekme kısmı
    a=[]
    b=[]
    for i in range(len(kriterler['bir'])):
        a.append(kriterler["bir"][i].text)
        b.append(kriterler["iki"][i].text)
    DT=(pd.DataFrame([b],columns=a)>>mutate(urls=url))
    KriterDF.append(DT)
        
        

In [6]:
kariyerDF=pd.DataFrame(list(zip(pozisyon,city,firma,metin,b_sayisi,g_sayisi,sure,urls)),
             columns =['pozisyon', 'city','firma','metin','b_sayisi','g_sayisi','sure','urls'])

In [8]:
kriterDF=pd.concat(KriterDF)

C:\Users\fatih\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [23]:
pd.merge(kariyerDF, kriterDF, how='inner', left_on='urls', right_on='urls').shape

pozisyon                 city  \
0           İhracat Uzmanı  Tekirdağ(Çerkezköy)   
1  Dijital İletişim Uzmanı                İzmir   
2          Bakım Sorumlusu              Kocaeli   

                                               firma  \
0  ÇEMOBSAN METAL VE PLASTİK SANAYİ TİCARET LİMİT...   
1  DOĞAN METALURJİ MAKİNA VE İNŞ TAAHÜT SANVE TİC...   
2                MİTSUBA TEKLAS TURKEY OTOMOTİV A.Ş.   

                                               metin b_sayisi g_sayisi sure  \
0  GENEL NİTELİKLER VE İŞ TANIMI\nÇemobsan Metal ...  100-200     1336  453   
1  GENEL NİTELİKLER VE İŞ TANIMI\nDijital İletişi...  200-300     1548  468   
2  GENEL NİTELİKLER VE İŞ TANIMI\n·      Gebze Or...  200-300     2103  468   

                                                urls Askerlik Durumu:  \
0  https://www.kariyer.net/is-ilani/cemobsan-meta...              NaN   
1  https://www.kariyer.net/is-ilani/dogan-metalur...              NaN   
2  https://www.kariyer.net/is-ilani/mitsuba-tekla...          Yapıldı   

          Departman:   Eğitim Seviyesi:  \
0    İthalat/İhracat  Üniversite(Mezun)   
1  Dijital Pazarlama  Üniversite(Mezun)   
2     Bakım / Onarım  Üniversite(Mezun)   

                                      Firma Sektörü:  Personel Sayısı:  \
0  Üretim / Endüstriyel Ürünler, Maden ve Metal S...  Belirtilmemiştir   
1                            Restaurant - Cafe - Bar  Belirtilmemiştir   
2                                           Otomotiv                 1   

  Pozisyon Seviyesi:                       Tecrübe:  \
0                NaN    1 - 3 yıl tecrübeli adaylar   
1                NaN  En az 3 yıl tecrübeli adaylar   
2              Uzman    1 - 3 yıl tecrübeli adaylar   

                                        Yabancı Dil:        Çalışma Şekli :  \
0  İngilizce( Okuma : İleri, Yazma : İleri, Konuş...  Sürekli / Tam zamanlı   
1  İngilizce( Okuma : İleri, Yazma : İleri, Konuş...  Sürekli / Tam zamanlı   
2                                                NaN  Sürekli / Tam zamanlı   

           Ülke/Şehir:     Üniversite Bölümü:  
0  Tekirdağ(Çerkezköy)                    NaN  
1                İzmir                    NaN  
2              Kocaeli  Elektrik Mühendisliği